In [93]:
import os
import pandas as pd
import openpyxl
from fuzzywuzzy import fuzz
import requests
import pandas as pd
from bs4 import BeautifulSoup
#import credentials
import re
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.utils.dataframe import dataframe_to_rows


In [64]:
def auto_adjust_columns(worksheet):
    for column_cells in worksheet.columns:
        length = max(len(str(cell.value)) for cell in column_cells)
        column_name = column_cells[0].column_letter
        worksheet.column_dimensions[column_name].width = length + 2  # +2 para agregar un pequeño margen


def has_12_or_13_digits(x):
    if not (isinstance(x, int) or isinstance(x, float)):
        return False

    try:
        return len(str(int(x))) in [12, 13]
    except ValueError:
        return False


def clean_proveedor(filename, new_filename=None):
    # Cargar el archivo de Excel
    df = pd.read_excel(filename)

    # Eliminar filas sin un código asociado
    columna = df.iloc[:, 1]
    df = df.dropna(subset=[df.columns[1]])

    # Calcular el porcentaje de valores nulos en cada columna
    null_percentage = df.isnull().mean() * 100

    # Encontrar las columnas con más del 90% de valores nulos
    cols_to_drop = null_percentage[null_percentage > 90].index

    # Eliminar las columnas con más del 90% de valores nulos
    df = df.drop(cols_to_drop, axis=1)

    # Eliminar las filas que tienen datos en menos de 3 celdas
    df = df.dropna(thresh=3)

    # Cambiar el nombre de las columnas que tienen al menos el 50% de números con 12 o 13 dígitos a "Codigo de Barra"
    for col in df.columns:
        num_of_rows = len(df[col].dropna())
        num_of_rows_with_12_or_13_digits = df[col].apply(has_12_or_13_digits).sum()

        if num_of_rows_with_12_or_13_digits / num_of_rows >= 0.5:
            df.rename(columns={col: "Codigo de Barra"}, inplace=True)

    df['Codigo de Barra'] = df['Codigo de Barra'].astype(str)
    df['Codigo de Barra'] = df['Codigo de Barra'].str.replace(r'[.+\-E]', '', regex=True)
    df['Codigo de Barra'] = df['Codigo de Barra'].str.zfill(13)

    # Cambiar el nombre de las columnas que tienen al menos el 30% de celdas con valores con coma o punto a "Costo"
    for col in df.columns:
        num_of_rows = len(df[col].dropna())
        num_of_rows_with_comma_or_point = df[col].astype(str).str.contains('[,.]').sum()

        if num_of_rows_with_comma_or_point / num_of_rows >= 0.3:
            df.rename(columns={col: "Costo"}, inplace=True)



    # Eliminar las columnas que tienen al menos el 90% de sus celdas iguales entre ellas
    for col in df.columns:
        num_of_rows = len(df[col].dropna())
        most_frequent_value_count = df[col].value_counts().iloc[0]

        if most_frequent_value_count / num_of_rows >= 0.9:
            df = df.drop(col, axis=1)

    # Cambiar el nombre de la columna que tiene la longitud promedio de caracteres más larga y una mezcla alfanumérica a "NOMBRE"
    max_avg_length = 0
    nombre_col = None

    for col in df.columns:
        col_values = df[col].dropna().astype(str)
        col_avg_length = col_values.str.len().mean()
        col_has_alphanumeric_mix = col_values.str.contains(r'[A-Za-z]+.*\d+|\d+.*[A-Za-z]+').any()

        if col_has_alphanumeric_mix and col_avg_length > max_avg_length:
            max_avg_length = col_avg_length
            nombre_col = col

    if nombre_col is not None:
        df.rename(columns={nombre_col: "Nombre"}, inplace=True)



    # Guardar el archivo de Excel modificado en la carpeta "procesados"
    processed_dir = "./procesados"
    os.makedirs(processed_dir, exist_ok=True)  # crea la carpeta si no existe

    if new_filename is None:
        new_filename = os.path.basename(filename)
    else:
        _, file_extension = os.path.splitext(filename)
        new_filename = new_filename + file_extension

    processed_filename = os.path.join(processed_dir, new_filename)
    df.to_excel(processed_filename, index=False)

    print('Archivo limpio guardado como', processed_filename)

    # Eliminar las formas que no son imágenes
    wb = openpyxl.load_workbook(processed_filename)
    ws = wb.active
    auto_adjust_columns(ws)  # Añade esta línea
    # Iterar sobre todas las formas en la hoja de cálculo
    try:
        # Iterar sobre todas las formas en la hoja de cálculo
        for shape in ws._shapes:
            # Eliminar la forma si no es un gráfico
            if not isinstance(shape, openpyxl.drawing.image.Image):
                ws.remove_shape(shape)
    except AttributeError:
        # Si no se puede acceder a _shapes, usar _images en su lugar
        for image in ws._images:
            # Eliminar la imagen
            ws.remove_image(image)

    wb.save(processed_filename)


In [65]:
clean_proveedor('./sinProcesar/algabo.xlsx','Algabo')

Archivo limpio guardado como ./procesados/Algabo.xlsx


In [66]:
clean_proveedor('./sinProcesar/teddy.xlsx','Teddy')

Archivo limpio guardado como ./procesados/Teddy.xlsx


In [67]:
clean_proveedor('./sinProcesar/upalala.xlsx','Upalala')

ValueError: Grouper for 'Costo' not 1-dimensional